# Capstone 2 Data Munging & Merging
***
The goal of this notebook is to merge and munge all of the data for Springboard's Second Capstone Project: Predicting Enrollment: Developing a County Centered Marketing Strategy 

In [1]:
# Importing tools
import pandas as pd
import numpy as np

The first data set I wish to create is the 2015 dataset.  This data will be used to add more observations to my machine learning model, or be used to test my model built with the 2016 data.

In [2]:
# Import All 2015 Datasets
dfs_2015 = []
datasets = ['by_Age_Group', 'by_APTC', 'by_CSR','by_Ethnicity','by_Household_Income_by_Percent_of_FPL',
           'by_Metal_Level','Consumer_Type']
for ds in datasets:
    dfs_2015.append(pd.read_csv('data/2015/2015_'+ds+'.csv'))

In [3]:
dfs_2015[0].head()

,County FIPS Code,State Name,County Name,0-17,18-25,26-34,35-44,45-54,55-64,65+,Unknown,Total Plan Selections
0,2013,AK,Aleutians East Borough,.,0,.,.,.,12,0,0,29
1,2016,AK,Aleutians West Census Area,.,.,.,.,14,.,.,0,34
2,2020,AK,Anchorage Municipality,869,864,1879,1393,1666,2010,67,0,8748
3,2050,AK,Bethel Census Area,.,.,13,15,16,24,0,0,83
4,2060,AK,Bristol Bay Borough,.,.,.,.,.,.,0,.,29


In [4]:
# Creating List of Column Names for DataFrames
age= ['cfc', 'state', 'county', '0-17','18-25','26-34','35-44','45-54','55-64','65+','age_unknown','tps']
aptc = ['cfc', 'state','county','no_aptc','yes_aptc','tps']
csr = ['cfc','state','county','no_csr','yes_csr','tps']
ethnicity = ['cfc','state','county','american_alaska_native','asian','black','latino','hawaiian_native_pacific_islander',
            'multiracial','white','ethnicity_unknown','tps']
fpl = ['cfc','state','county','fpl_<_100','fpl_<_151','fpl_<_201','fpl_<_251','fpl_<_301','fpl_<_401', 'fpl_>_400',
      'fpl_unknown','tps']
metal = ['cfc','state','county','platinum','gold','silver','bronze','catastrophic','tps']
c_type = ['cfc','state','county','new','auto_renew','active_renew','tps']

# Edit Column Names for DataFrames
# List of Lists
df_list = [age,aptc,csr,ethnicity,fpl,metal,c_type]
# List Starting Point
n = 0
# Loop Through DataFrames to Update Column Names
for i in range(7):
    dfs_2015[i].columns = df_list[n]
    n+=1

In [5]:
dfs_2015[0].head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,age_unknown,tps
0,2013,AK,Aleutians East Borough,.,0,.,.,.,12,0,0,29
1,2016,AK,Aleutians West Census Area,.,.,.,.,14,.,.,0,34
2,2020,AK,Anchorage Municipality,869,864,1879,1393,1666,2010,67,0,8748
3,2050,AK,Bethel Census Area,.,.,13,15,16,24,0,0,83
4,2060,AK,Bristol Bay Borough,.,.,.,.,.,.,0,.,29


In [6]:
# Dropping All Total Plan Selection ('tps') Columns Except in Metal Tier DataFrame
for i in range(5):
    dfs_2015[i] = dfs_2015[i].drop('tps', axis=1)
dfs_2015[6].drop('tps',axis=1, inplace=True)

In [7]:
dfs_2015[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2599 entries, 0 to 2598
Data columns (total 9 columns):
cfc             2599 non-null object
state           2599 non-null object
county          2599 non-null object
platinum        2599 non-null object
gold            2599 non-null object
silver          2599 non-null object
bronze          2599 non-null object
catastrophic    2599 non-null object
tps             2599 non-null object
dtypes: object(9)
memory usage: 182.8+ KB


In [8]:
# Merge Dataframes into Sigle Dataframe.
master_2015 = dfs_2015[0]

for df in dfs_2015[1:]:
    master_2015 = master_2015.merge(df, on=['cfc','state','county'])

In [9]:
master_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2413 entries, 0 to 2412
Data columns (total 40 columns):
cfc                                 2413 non-null object
state                               2413 non-null object
county                              2413 non-null object
0-17                                2413 non-null object
18-25                               2413 non-null object
26-34                               2413 non-null object
35-44                               2413 non-null object
45-54                               2413 non-null object
55-64                               2413 non-null object
65+                                 2413 non-null object
age_unknown                         2413 non-null object
no_aptc                             2218 non-null float64
yes_aptc                            2216 non-null float64
no_csr                              2413 non-null object
yes_csr                             2413 non-null object
american_alaska_native              

Now it's time to munge the data.  Null values are represented as NaN values or '.' values.  My plan is to convert all '.' values first, then see how many NaN values each observation has.  I think some NaN values can be converted to 0, as there are plenty of legitimate 0s within the data.  However, if an observation has more than 10 NaNs, I don't believe it will be beneficial to the prediction model.  As such, I will set a dropna threshold of 10.

In [10]:
# Master DF Before Coercing to Numeric Values
master_2015.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,tps,new,auto_renew,active_renew
0,12109,FL,St. Johns County,1259,1093,1566,1684,2354,2963,41,...,599,1128,594,7209,1967,62,10960,5875.0,1302.0,3783.0
1,10001,DE,Kent County,413,284,529,530,771,872,18,...,172,107,528,2136,629,17,3417,1853.0,909.0,655.0
2,10003,DE,New Castle County,1964,1275,2487,2333,3288,3494,126,...,1346,679,2406,8835,2943,104,14967,7453.0,4174.0,3340.0
3,10005,DE,Sussex County,794,436,805,929,1453,2204,30,...,466,257,1144,4198,1032,20,6651,3408.0,1712.0,1531.0
4,12001,FL,Alachua County,726,2161,2734,1867,2250,2940,41,...,523,1188,429,9169,1848,85,12719,6443.0,1898.0,4378.0


In [11]:
master_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2413 entries, 0 to 2412
Data columns (total 40 columns):
cfc                                 2413 non-null object
state                               2413 non-null object
county                              2413 non-null object
0-17                                2413 non-null object
18-25                               2413 non-null object
26-34                               2413 non-null object
35-44                               2413 non-null object
45-54                               2413 non-null object
55-64                               2413 non-null object
65+                                 2413 non-null object
age_unknown                         2413 non-null object
no_aptc                             2218 non-null float64
yes_aptc                            2216 non-null float64
no_csr                              2413 non-null object
yes_csr                             2413 non-null object
american_alaska_native              

In [12]:
# Coerce Numbered Categories to Numberic to get NaNs
tonumeric=['0-17','18-25','26-34','35-44','45-54','55-64','65+','age_unknown','no_aptc','yes_aptc','no_csr','yes_csr',
           'american_alaska_native','asian','black','latino','hawaiian_native_pacific_islander','multiracial','white',
           'ethnicity_unknown','fpl_<_100','fpl_<_151','fpl_<_201','fpl_<_251','fpl_<_301','fpl_<_401', 'fpl_>_400',
           'fpl_unknown','platinum','gold','silver','bronze','catastrophic','new','auto_renew','active_renew','tps']
dfa = master_2015[tonumeric].apply(pd.to_numeric, errors='coerce')
#Setting up additional columns to concatinate
dfb = master_2015[['cfc','state','county']]

In [13]:
master_2015 = pd.concat([dfb,dfa], axis=1)
master_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2413 entries, 0 to 2412
Data columns (total 40 columns):
cfc                                 2413 non-null object
state                               2413 non-null object
county                              2413 non-null object
0-17                                1362 non-null float64
18-25                               1729 non-null float64
26-34                               2259 non-null float64
35-44                               2269 non-null float64
45-54                               2349 non-null float64
55-64                               2388 non-null float64
65+                                 957 non-null float64
age_unknown                         2255 non-null float64
no_aptc                             2218 non-null float64
yes_aptc                            2216 non-null float64
no_csr                              2392 non-null float64
yes_csr                             2392 non-null float64
american_alaska_native     

In [14]:
# Master DF After Numeric Conversion
master_2015.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,new,auto_renew,active_renew,tps
0,12109,FL,St. Johns County,1259.0,1093.0,1566.0,1684.0,2354.0,2963.0,41.0,...,599.0,1128.0,594.0,7209.0,1967.0,62.0,5875.0,1302.0,3783.0,10960.0
1,10001,DE,Kent County,413.0,284.0,529.0,530.0,771.0,872.0,18.0,...,172.0,107.0,528.0,2136.0,629.0,17.0,1853.0,909.0,655.0,3417.0
2,10003,DE,New Castle County,1964.0,1275.0,2487.0,2333.0,3288.0,3494.0,126.0,...,1346.0,679.0,2406.0,8835.0,2943.0,104.0,7453.0,4174.0,3340.0,14967.0
3,10005,DE,Sussex County,794.0,436.0,805.0,929.0,1453.0,2204.0,30.0,...,466.0,257.0,1144.0,4198.0,1032.0,20.0,3408.0,1712.0,1531.0,6651.0
4,12001,FL,Alachua County,726.0,2161.0,2734.0,1867.0,2250.0,2940.0,41.0,...,523.0,1188.0,429.0,9169.0,1848.0,85.0,6443.0,1898.0,4378.0,12719.0


In [15]:
# Dropping Observations with at Least 10 NaN Values
master_2015.dropna(axis=0, thresh=30, inplace=True)
master_2015.reset_index(drop=True, inplace=True)
master_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 40 columns):
cfc                                 2109 non-null object
state                               2109 non-null object
county                              2109 non-null object
0-17                                1272 non-null float64
18-25                               1632 non-null float64
26-34                               2066 non-null float64
35-44                               2081 non-null float64
45-54                               2102 non-null float64
55-64                               2107 non-null float64
65+                                 804 non-null float64
age_unknown                         1961 non-null float64
no_aptc                             2061 non-null float64
yes_aptc                            2059 non-null float64
no_csr                              2107 non-null float64
yes_csr                             2107 non-null float64
american_alaska_native     

In [16]:
master_2015.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,new,auto_renew,active_renew,tps
0,12109,FL,St. Johns County,1259.0,1093.0,1566.0,1684.0,2354.0,2963.0,41.0,...,599.0,1128.0,594.0,7209.0,1967.0,62.0,5875.0,1302.0,3783.0,10960.0
1,10001,DE,Kent County,413.0,284.0,529.0,530.0,771.0,872.0,18.0,...,172.0,107.0,528.0,2136.0,629.0,17.0,1853.0,909.0,655.0,3417.0
2,10003,DE,New Castle County,1964.0,1275.0,2487.0,2333.0,3288.0,3494.0,126.0,...,1346.0,679.0,2406.0,8835.0,2943.0,104.0,7453.0,4174.0,3340.0,14967.0
3,10005,DE,Sussex County,794.0,436.0,805.0,929.0,1453.0,2204.0,30.0,...,466.0,257.0,1144.0,4198.0,1032.0,20.0,3408.0,1712.0,1531.0,6651.0
4,12001,FL,Alachua County,726.0,2161.0,2734.0,1867.0,2250.0,2940.0,41.0,...,523.0,1188.0,429.0,9169.0,1848.0,85.0,6443.0,1898.0,4378.0,12719.0


In [17]:
#Convert NaN Values to 0
master_2015.fillna(0, inplace=True)

master_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109 entries, 0 to 2108
Data columns (total 40 columns):
cfc                                 2109 non-null object
state                               2109 non-null object
county                              2109 non-null object
0-17                                2109 non-null float64
18-25                               2109 non-null float64
26-34                               2109 non-null float64
35-44                               2109 non-null float64
45-54                               2109 non-null float64
55-64                               2109 non-null float64
65+                                 2109 non-null float64
age_unknown                         2109 non-null float64
no_aptc                             2109 non-null float64
yes_aptc                            2109 non-null float64
no_csr                              2109 non-null float64
yes_csr                             2109 non-null float64
american_alaska_native    

In [18]:
master_2015.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,new,auto_renew,active_renew,tps
0,12109,FL,St. Johns County,1259.0,1093.0,1566.0,1684.0,2354.0,2963.0,41.0,...,599.0,1128.0,594.0,7209.0,1967.0,62.0,5875.0,1302.0,3783.0,10960.0
1,10001,DE,Kent County,413.0,284.0,529.0,530.0,771.0,872.0,18.0,...,172.0,107.0,528.0,2136.0,629.0,17.0,1853.0,909.0,655.0,3417.0
2,10003,DE,New Castle County,1964.0,1275.0,2487.0,2333.0,3288.0,3494.0,126.0,...,1346.0,679.0,2406.0,8835.0,2943.0,104.0,7453.0,4174.0,3340.0,14967.0
3,10005,DE,Sussex County,794.0,436.0,805.0,929.0,1453.0,2204.0,30.0,...,466.0,257.0,1144.0,4198.0,1032.0,20.0,3408.0,1712.0,1531.0,6651.0
4,12001,FL,Alachua County,726.0,2161.0,2734.0,1867.0,2250.0,2940.0,41.0,...,523.0,1188.0,429.0,9169.0,1848.0,85.0,6443.0,1898.0,4378.0,12719.0


In [19]:
# Saving the File
master_2015.to_csv('2015_Cleaned.csv')

### Repeating the Process for 2016 Data
***

In [20]:
# Import All 2015 Datasets
dfs_2016 = []
datasets = ['by_Age_Group', 'by_APTC', 'by_CSR','by_Ethnicity','by_Household_Income_by_Percent_of_FPL',
           'by_Metal_Level','Consumer_Type']
for ds in datasets:
    dfs_2016.append(pd.read_csv('data/2016/2016_'+ds+'.csv'))

In [21]:
dfs_2016[0].head()

,County FIPS Code,State Name,County Name,0-17,18-25,26-34,35-44,45-54,55-64,65+,Unknown,Total Plan Selections
0,02013,AK,Aleutians East Borough,.,.,.,.,.,16,0,0,38
1,02016,AK,Aleutians West Census Area,.,.,.,.,15,.,0,0,38
2,02020,AK,Anchorage Municipality,1117,828,1951,1502,1770,2194,122,0,9484
3,02050,AK,Bethel Census Area,11,.,.,.,12,29,0,0,73
4,02060,AK,Bristol Bay Borough,.,.,.,.,.,.,0,0,34


In [22]:
# Update Column Names Where Needed
fpl = ['cfc','state','county','fpl_<_100','fpl_<_139','fpl_<_151','fpl_<_201','fpl_<_251','fpl_<_301','fpl_<_401', 'fpl_>_400',
      'fpl_unknown','tps']
c_type = ['cfc','state','county','new','active_renew','auto_renew','tps']

In [23]:
# Edit Column Names for DataFrames
# Recreate List of Lists with Updated Info
df_list = [age,aptc,csr,ethnicity,fpl,metal,c_type]
# List Starting Point
n = 0
# Loop Through DataFrames to Update Column Names
for i in range(7):
    dfs_2016[i].columns = df_list[n]
    n+=1

In [24]:
dfs_2016[0].head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,age_unknown,tps
0,02013,AK,Aleutians East Borough,.,.,.,.,.,16,0,0,38
1,02016,AK,Aleutians West Census Area,.,.,.,.,15,.,0,0,38
2,02020,AK,Anchorage Municipality,1117,828,1951,1502,1770,2194,122,0,9484
3,02050,AK,Bethel Census Area,11,.,.,.,12,29,0,0,73
4,02060,AK,Bristol Bay Borough,.,.,.,.,.,.,0,0,34


In [25]:
# Dropping All Total Plan Selection ('tps') Columns Except in Last DataFrame
for i in range(5):
    dfs_2016[i] = dfs_2016[i].drop('tps', axis=1)
dfs_2016[6].drop('tps',axis=1, inplace=True)

In [26]:
dfs_2016[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 9 columns):
cfc             2602 non-null object
state           2602 non-null object
county          2602 non-null object
platinum        2603 non-null object
gold            2603 non-null object
silver          2603 non-null object
bronze          2603 non-null object
catastrophic    2603 non-null object
tps             2603 non-null object
dtypes: object(9)
memory usage: 183.1+ KB


In [27]:
# Merge Dataframes into Sigle Dataframe.
master_2016 = dfs_2016[0]

for df in dfs_2016[1:]:
    master_2016 = master_2016.merge(df, on=['cfc','state','county'])

In [28]:
master_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2603 entries, 0 to 2602
Data columns (total 41 columns):
cfc                                 2602 non-null object
state                               2602 non-null object
county                              2602 non-null object
0-17                                2603 non-null object
18-25                               2603 non-null object
26-34                               2603 non-null object
35-44                               2603 non-null object
45-54                               2603 non-null object
55-64                               2603 non-null object
65+                                 2603 non-null object
age_unknown                         2603 non-null object
no_aptc                             2603 non-null object
yes_aptc                            2603 non-null object
no_csr                              2603 non-null object
yes_csr                             2603 non-null object
american_alaska_native              26

In [29]:
# Master DF Before Numeric Coercion
master_2016.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,tps,new,active_renew,auto_renew
0,02013,AK,Aleutians East Borough,.,.,.,.,.,16,0,...,.,0,0,21,17,0,38,.,.,19
1,02016,AK,Aleutians West Census Area,.,.,.,.,15,.,0,...,.,0,.,20,17,.,38,18,.,.
2,02020,AK,Anchorage Municipality,1117,828,1951,1502,1770,2194,122,...,686,0,334,5038,4055,57,9484,3505,3744,2235
3,02050,AK,Bethel Census Area,11,.,.,.,12,29,0,...,.,0,.,21,51,.,73,17,16,40
4,02060,AK,Bristol Bay Borough,.,.,.,.,.,.,0,...,.,0,.,19,.,.,34,.,16,.


In [30]:
# Coerce Numbered Categories to Numberic to get NaNs
tonumeric=['0-17','18-25','26-34','35-44','45-54','55-64','65+','age_unknown','no_aptc','yes_aptc','no_csr','yes_csr',
           'american_alaska_native','asian','black','latino','hawaiian_native_pacific_islander','multiracial','white',
           'ethnicity_unknown','fpl_<_100','fpl_<_139','fpl_<_151','fpl_<_201','fpl_<_251','fpl_<_301','fpl_<_401', 'fpl_>_400',
           'fpl_unknown','platinum','gold','silver','bronze','catastrophic','new','active_renew','auto_renew','tps']
dfa = master_2016[tonumeric].apply(pd.to_numeric, errors='coerce')
#Setting up additional columns to concatinate
dfb = master_2016[['cfc','state','county']]

In [31]:
master_2016 = pd.concat([dfb,dfa], axis=1)
master_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2603 entries, 0 to 2602
Data columns (total 41 columns):
cfc                                 2602 non-null object
state                               2602 non-null object
county                              2602 non-null object
0-17                                2366 non-null float64
18-25                               2403 non-null float64
26-34                               2496 non-null float64
35-44                               2507 non-null float64
45-54                               2546 non-null float64
55-64                               2580 non-null float64
65+                                 1017 non-null float64
age_unknown                         1212 non-null float64
no_aptc                             2460 non-null float64
yes_aptc                            2460 non-null float64
no_csr                              2577 non-null float64
yes_csr                             2577 non-null float64
american_alaska_native    

In [32]:
# Master DF After Numeric Coercion
master_2016.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,new,active_renew,auto_renew,tps
0,02013,AK,Aleutians East Borough,NaN,NaN,NaN,NaN,NaN,16.0,0.0,...,NaN,0.0,0.0,21.0,17.0,0.0,NaN,NaN,19.0,38.0
1,02016,AK,Aleutians West Census Area,NaN,NaN,NaN,NaN,15.0,NaN,0.0,...,NaN,0.0,NaN,20.0,17.0,NaN,18.0,NaN,NaN,38.0
2,02020,AK,Anchorage Municipality,1117.0,828.0,1951.0,1502.0,1770.0,2194.0,122.0,...,686.0,0.0,334.0,5038.0,4055.0,57.0,3505.0,3744.0,2235.0,9484.0
3,02050,AK,Bethel Census Area,11.0,NaN,NaN,NaN,12.0,29.0,0.0,...,NaN,0.0,NaN,21.0,51.0,NaN,17.0,16.0,40.0,73.0
4,02060,AK,Bristol Bay Borough,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,NaN,19.0,NaN,NaN,NaN,16.0,NaN,34.0


In [33]:
# Dropping Observations with at Least 10 NaN Values
master_2016.dropna(axis=0, thresh=30, inplace=True)
master_2016.reset_index(drop=True, inplace=True)
master_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2428 entries, 0 to 2427
Data columns (total 41 columns):
cfc                                 2428 non-null object
state                               2427 non-null object
county                              2427 non-null object
0-17                                2314 non-null float64
18-25                               2357 non-null float64
26-34                               2408 non-null float64
35-44                               2418 non-null float64
45-54                               2424 non-null float64
55-64                               2426 non-null float64
65+                                 918 non-null float64
age_unknown                         1051 non-null float64
no_aptc                             2389 non-null float64
yes_aptc                            2389 non-null float64
no_csr                              2428 non-null float64
yes_csr                             2428 non-null float64
american_alaska_native     

In [34]:
master_2016.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,new,active_renew,auto_renew,tps
0,02020,AK,Anchorage Municipality,1117.0,828.0,1951.0,1502.0,1770.0,2194.0,122.0,...,686.0,0.0,334.0,5038.0,4055.0,57.0,3505.0,3744.0,2235.0,9484.0
1,02068,AK,Denali Borough,11.0,NaN,39.0,25.0,29.0,51.0,NaN,...,15.0,0.0,NaN,75.0,84.0,NaN,41.0,97.0,26.0,164.0
2,02090,AK,Fairbanks North Star Borough,315.0,199.0,537.0,379.0,411.0,529.0,12.0,...,193.0,0.0,107.0,1323.0,935.0,17.0,872.0,1005.0,505.0,2382.0
3,02100,AK,Haines Borough,31.0,16.0,57.0,41.0,61.0,79.0,0.0,...,13.0,0.0,NaN,122.0,159.0,NaN,83.0,139.0,63.0,285.0
4,02110,AK,Juneau City and Borough,146.0,84.0,309.0,178.0,178.0,181.0,NaN,...,80.0,0.0,NaN,507.0,520.0,NaN,386.0,444.0,252.0,1082.0


In [35]:
#Convert NaN Values to 0
master_2016.fillna(0, inplace=True)

master_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2428 entries, 0 to 2427
Data columns (total 41 columns):
cfc                                 2428 non-null object
state                               2428 non-null object
county                              2428 non-null object
0-17                                2428 non-null float64
18-25                               2428 non-null float64
26-34                               2428 non-null float64
35-44                               2428 non-null float64
45-54                               2428 non-null float64
55-64                               2428 non-null float64
65+                                 2428 non-null float64
age_unknown                         2428 non-null float64
no_aptc                             2428 non-null float64
yes_aptc                            2428 non-null float64
no_csr                              2428 non-null float64
yes_csr                             2428 non-null float64
american_alaska_native    

In [36]:
master_2016.tail()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,new,active_renew,auto_renew,tps
2423,56039,WY,Teton County,301.0,225.0,901.0,496.0,438.0,449.0,0.0,...,182.0,0.0,86.0,1810.0,905.0,11.0,996.0,1394.0,422.0,2812.0
2424,56041,WY,Uinta County,120.0,47.0,102.0,110.0,104.0,179.0,0.0,...,32.0,0.0,34.0,454.0,164.0,11.0,344.0,235.0,84.0,663.0
2425,56043,WY,Washakie County,48.0,33.0,36.0,60.0,55.0,107.0,0.0,...,16.0,0.0,0.0,235.0,90.0,0.0,146.0,144.0,51.0,341.0
2426,56045,WY,Weston County,27.0,13.0,23.0,30.0,53.0,91.0,0.0,...,26.0,0.0,0.0,154.0,80.0,0.0,114.0,104.0,19.0,237.0
2427,Unsuppressed Total,0,0,884172.0,1067477.0,1608390.0,1555651.0,2010657.0,2431625.0,67954.0,...,740152.0,71701.0,571327.0,6823481.0,2060447.0,99026.0,4025637.0,3918454.0,1681891.0,9625982.0


In [37]:
# Deleting "Unsuppressed Total" Observation Row
master_2016.drop(master_2016.index[2427], inplace=True)
master_2016.tail()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,fpl_unknown,platinum,gold,silver,bronze,catastrophic,new,active_renew,auto_renew,tps
2422,56037,WY,Sweetwater County,148.0,106.0,222.0,169.0,213.0,308.0,0.0,...,76.0,0.0,46.0,841.0,266.0,15.0,602.0,433.0,133.0,1168.0
2423,56039,WY,Teton County,301.0,225.0,901.0,496.0,438.0,449.0,0.0,...,182.0,0.0,86.0,1810.0,905.0,11.0,996.0,1394.0,422.0,2812.0
2424,56041,WY,Uinta County,120.0,47.0,102.0,110.0,104.0,179.0,0.0,...,32.0,0.0,34.0,454.0,164.0,11.0,344.0,235.0,84.0,663.0
2425,56043,WY,Washakie County,48.0,33.0,36.0,60.0,55.0,107.0,0.0,...,16.0,0.0,0.0,235.0,90.0,0.0,146.0,144.0,51.0,341.0
2426,56045,WY,Weston County,27.0,13.0,23.0,30.0,53.0,91.0,0.0,...,26.0,0.0,0.0,154.0,80.0,0.0,114.0,104.0,19.0,237.0


In [38]:
# Saving the File
master_2016.to_csv('2016_Cleaned.csv')

***
To improve my prediciton models, I thought combining the two datasets together would nearly double the amount of test data available.  I will also need to combine the FPL<139 column into the FPL<151 column.  This is due to the FPL<138 column not being present in the 2015 dataset.  I will handle this by converting the Null Values to Zero, adding the two columns together, and then dropping the FPL<139 column alltogether. 

In [39]:
# Add Year Column to the Data
master_2015['Year'] = 2015
master_2016['Year'] = 2016


In [40]:
# Merge Cleaned Dataframes Together
master = pd.concat([master_2015,master_2016], axis=0)
master = master[master_2016.columns]

In [41]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4536 entries, 0 to 2426
Data columns (total 42 columns):
cfc                                 4536 non-null object
state                               4536 non-null object
county                              4536 non-null object
0-17                                4536 non-null float64
18-25                               4536 non-null float64
26-34                               4536 non-null float64
35-44                               4536 non-null float64
45-54                               4536 non-null float64
55-64                               4536 non-null float64
65+                                 4536 non-null float64
age_unknown                         4536 non-null float64
no_aptc                             4536 non-null float64
yes_aptc                            4536 non-null float64
no_csr                              4536 non-null float64
yes_csr                             4536 non-null float64
american_alaska_native    

In [42]:
#Convert FPL < 139 NaN Values to 0 for the 2015 Da
master.fillna(0, inplace=True)

In [43]:
# Add FPL<139 Column Into FPL<151 column
x = pd.Series(x for x in master['fpl_<_139'])
y = pd.Series(y for y in master['fpl_<_151'])
combined = list(x+y)

# Check Length to Confirm Addition Over Concatenation
len(combined)

4536

In [44]:
# Replace FPL<151 Column and Drop FPL<139 Column
master['fpl_<_151'] = combined
master.drop('fpl_<_139', axis=1, inplace=True)
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4536 entries, 0 to 2426
Data columns (total 41 columns):
cfc                                 4536 non-null object
state                               4536 non-null object
county                              4536 non-null object
0-17                                4536 non-null float64
18-25                               4536 non-null float64
26-34                               4536 non-null float64
35-44                               4536 non-null float64
45-54                               4536 non-null float64
55-64                               4536 non-null float64
65+                                 4536 non-null float64
age_unknown                         4536 non-null float64
no_aptc                             4536 non-null float64
yes_aptc                            4536 non-null float64
no_csr                              4536 non-null float64
yes_csr                             4536 non-null float64
american_alaska_native    

After the first machine learning models gave back very weak R^2.  One way to improve our model is to convert tier counts to percent of total plan selections.

In [45]:
master.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,platinum,gold,silver,bronze,catastrophic,new,active_renew,auto_renew,tps,Year
0,12109,FL,St. Johns County,1259.0,1093.0,1566.0,1684.0,2354.0,2963.0,41.0,...,1128.0,594.0,7209.0,1967.0,62.0,5875.0,3783.0,1302.0,10960.0,2015
1,10001,DE,Kent County,413.0,284.0,529.0,530.0,771.0,872.0,18.0,...,107.0,528.0,2136.0,629.0,17.0,1853.0,655.0,909.0,3417.0,2015
2,10003,DE,New Castle County,1964.0,1275.0,2487.0,2333.0,3288.0,3494.0,126.0,...,679.0,2406.0,8835.0,2943.0,104.0,7453.0,3340.0,4174.0,14967.0,2015
3,10005,DE,Sussex County,794.0,436.0,805.0,929.0,1453.0,2204.0,30.0,...,257.0,1144.0,4198.0,1032.0,20.0,3408.0,1531.0,1712.0,6651.0,2015
4,12001,FL,Alachua County,726.0,2161.0,2734.0,1867.0,2250.0,2940.0,41.0,...,1188.0,429.0,9169.0,1848.0,85.0,6443.0,4378.0,1898.0,12719.0,2015


In [46]:
# Saving the File
master.to_csv('Combo_Cleaned.csv')

In [47]:
# Create List of Tiers
tiers =['catastrophic','bronze','silver','gold','platinum']

# Convert Total Plan Selections to a Pandas Series
y = pd.Series(y for y in master['tps'])

# Iterate through tier features and divide them from the total plan selections to get percent of total.
for tier in tiers:
    x = pd.Series(x for x in master[tier])
    percent = list(x/y)
    master[tier] = percent
    master[tier] = master[tier].round(2)

In [48]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4536 entries, 0 to 2426
Data columns (total 41 columns):
cfc                                 4536 non-null object
state                               4536 non-null object
county                              4536 non-null object
0-17                                4536 non-null float64
18-25                               4536 non-null float64
26-34                               4536 non-null float64
35-44                               4536 non-null float64
45-54                               4536 non-null float64
55-64                               4536 non-null float64
65+                                 4536 non-null float64
age_unknown                         4536 non-null float64
no_aptc                             4536 non-null float64
yes_aptc                            4536 non-null float64
no_csr                              4536 non-null float64
yes_csr                             4536 non-null float64
american_alaska_native    

In [49]:
#  Review Null Line
master[master['catastrophic'].isnull()]

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,platinum,gold,silver,bronze,catastrophic,new,active_renew,auto_renew,tps,Year
1717,48269,TX,King County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2015
1730,48301,TX,Loving County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2015


In [50]:
master.drop(master.index[[1717,1730]], inplace=True)

In [51]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4532 entries, 0 to 2426
Data columns (total 41 columns):
cfc                                 4532 non-null object
state                               4532 non-null object
county                              4532 non-null object
0-17                                4532 non-null float64
18-25                               4532 non-null float64
26-34                               4532 non-null float64
35-44                               4532 non-null float64
45-54                               4532 non-null float64
55-64                               4532 non-null float64
65+                                 4532 non-null float64
age_unknown                         4532 non-null float64
no_aptc                             4532 non-null float64
yes_aptc                            4532 non-null float64
no_csr                              4532 non-null float64
yes_csr                             4532 non-null float64
american_alaska_native    

In [52]:
master.to_csv('Combo_Percents.csv')

Converting the health plan tiers (target values) to percents of total plan selections appeared to help the overall R^2 of the model.  However, the model still needs to be improved.  Converting all values to percent of target value in order to further scale the model.  

In [53]:
# Create list of features to convert to percents
topercent=['0-17','18-25','26-34','35-44','45-54','55-64','65+','age_unknown','no_aptc','yes_aptc','no_csr','yes_csr',
           'american_alaska_native','asian','black','latino','hawaiian_native_pacific_islander','multiracial','white',
           'ethnicity_unknown','fpl_<_100','fpl_<_151','fpl_<_201','fpl_<_251','fpl_<_301','fpl_<_401', 'fpl_>_400',
           'fpl_unknown','new','active_renew','auto_renew']

# Reconvert Total Plan Selections to a Pandas Series as total number of observations have changed.
y = pd.Series(y for y in master['tps'])

# Iterate through features and divide them from the total plan selections to get percent of total.
for feature in topercent:
    x = pd.Series(x for x in master[feature])
    percent = list(x/y)
    master[feature] = percent
    master[feature] = master[feature].round(4)

In [54]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4532 entries, 0 to 2426
Data columns (total 41 columns):
cfc                                 4532 non-null object
state                               4532 non-null object
county                              4532 non-null object
0-17                                4532 non-null float64
18-25                               4532 non-null float64
26-34                               4532 non-null float64
35-44                               4532 non-null float64
45-54                               4532 non-null float64
55-64                               4532 non-null float64
65+                                 4532 non-null float64
age_unknown                         4532 non-null float64
no_aptc                             4532 non-null float64
yes_aptc                            4532 non-null float64
no_csr                              4532 non-null float64
yes_csr                             4532 non-null float64
american_alaska_native    

In [55]:
master.head()

,cfc,state,county,0-17,18-25,26-34,35-44,45-54,55-64,65+,...,platinum,gold,silver,bronze,catastrophic,new,active_renew,auto_renew,tps,Year
0,12109,FL,St. Johns County,0.1149,0.0997,0.1429,0.1536,0.2148,0.2703,0.0037,...,0.10,0.05,0.66,0.18,0.01,0.5360,0.3452,0.1188,10960.0,2015
1,10001,DE,Kent County,0.1209,0.0831,0.1548,0.1551,0.2256,0.2552,0.0053,...,0.03,0.15,0.63,0.18,0.00,0.5423,0.1917,0.2660,3417.0,2015
2,10003,DE,New Castle County,0.1312,0.0852,0.1662,0.1559,0.2197,0.2334,0.0084,...,0.05,0.16,0.59,0.20,0.01,0.4980,0.2232,0.2789,14967.0,2015
3,10005,DE,Sussex County,0.1194,0.0656,0.1210,0.1397,0.2185,0.3314,0.0045,...,0.04,0.17,0.63,0.16,0.00,0.5124,0.2302,0.2574,6651.0,2015
4,12001,FL,Alachua County,0.0571,0.1699,0.2150,0.1468,0.1769,0.2312,0.0032,...,0.09,0.03,0.72,0.15,0.01,0.5066,0.3442,0.1492,12719.0,2015


In [56]:
master.to_csv('Combo_All_Percents.csv')